In [ ]:
%run RNN_Train_Horizon.ipynb

# Get Data

In [ ]:
Stock_Code = 'VOO'

In [ ]:
AnalysisPath =  f"{RNN_PATH}/{Stock_Code}/Analysis"

# Create directory if it doesn't exist
if not os.path.exists(AnalysisPath):

    os.makedirs(AnalysisPath)

print(f"Created an Analysis Folder for this! ")


In [ ]:
path = f'{SOURCE_PATH}/{Stock_Code}.csv'
df = pd.read_csv(path)[['Code','Date','Close']]
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date', ascending=True).reset_index(drop=True)

print(f"There are {len(df)} days in total ")

# Parameter List

In [ ]:
# 1 : Add Incrementally
# 2 : Divided by Number of Group
LoopMode = 2

## destination 

In [ ]:
today = datetime.now().strftime("%Y%m%d")


target_path = f"{RNN_PATH}/{Stock_Code}/Analysis"
csv_path = f"{target_path}/Model_Result.csv"


In [ ]:
if os.path.exists(target_path) and len(os.listdir(target_path)) > 0:

    backup_path = f"{RNN_PATH}/{Stock_Code}/Analysis/BackUp/{today}" 
    
    if not os.path.exists(backup_path):

        os.makedirs(backup_path)    

    [shutil.move(os.path.join(target_path, item), backup_path) for item in os.listdir(target_path) if item != 'BackUp']

## Variables affecting the model result

### Initial Value

In [ ]:
df_len = 200       # Length of data source
t_prop = 6         # proportion of training data
layer = 2          # Model layer of neural network
horizon = 2        # Number of prediction for each

past = horizon * 2 # How many days of past data

### Define the maximum Value of each variables

In [ ]:
Max_df_len = 2000  
Max_t_prop = 7     
Max_layer = 3
Max_horizon = 20

### Loop Option 1 - Define Incremental + Maximum of variables

In [ ]:
Inc_df_len = 400   
Inc_t_prop = 2
Inc_layer = 2
Inc_horizon = 10

### Loop Option 2 - Define Number of Group and update the incremental values

In [ ]:
Grp_past = 2

if LoopMode == 2:

    Grp_df_len = 3  
    Grp_t_prop = 2
    Grp_layer = 1
    Grp_horizon = 5
   
    
    Inc_df_len = int((Max_df_len - df_len)/Grp_df_len) + 1
    Inc_t_prop = int((Max_t_prop - t_prop)/Grp_t_prop) + 1
    Inc_layer = int((Max_layer - layer)/Grp_layer) + 1
    Inc_horizon = int((Max_horizon - horizon)/Grp_horizon) + 1


# Count total combination

In [ ]:
import math

print( "Total Combination = ",  (
        math.ceil((Max_df_len + 1 - df_len) / Inc_df_len) *    #3
        math.ceil((Max_t_prop + 1 - t_prop) / Inc_t_prop) *    #2
        math.ceil((Max_layer  + 1 - layer) / Inc_layer)  *     #2
        math.ceil((Max_horizon+ 1 - horizon) / Inc_horizon) *  #3
        Grp_past
    )
)

# Function to save predictions result

In [ ]:
def Save_Prediction(predict_result):
    
    predict_result = pd.DataFrame(predict_result)

    if os.path.exists(csv_path):
        
        new_data = pd.read_csv(csv_path) 
        predict_result = pd.concat([new_data , predict_result], ignore_index = True) 
    
    
    predict_result.to_csv(csv_path, index = False)


# Main Loop

In [ ]:
Full_Result = []
id = 1
isRun = True


for d in range( df_len, Max_df_len + 1 ,  Inc_df_len):

    input_df = df[-d:].copy()
    
    Max_past = int(len(input_df) / 4)
    Inc_past = int((Max_past - past)/Grp_past) 

    
    for t in range(t_prop, Max_t_prop + 1, Inc_t_prop):

        t = t / 10

        for l in range(layer, Max_layer  + 1, Inc_layer):

            for h in range(horizon, Max_horizon  + 1, Inc_horizon):

                for p in range( past, Max_past + 1, Inc_past):

                    intro_str = f"Data Len = {str(d).ljust(4)}, Train = {t}, Layer = {l}, Horizon = {str(h).ljust(2)}, Past_Length = {str(p).ljust(4)}"

                    print(f"{(str(id) + ')').ljust(5)} Running Data with {intro_str} ")   

                    
                    
                    if d > (p + h) * 2 :
                    
                        # Run RNN
                        mae, mae_model, model, data_df, indi_df = RNN(input_df, p , t , h, id , d)

                        
                        # Title and model name
                        RefName = f"RNN_{Stock_Code}_H{str(h).zfill(len(str(Max_horizon)))}_{id}"                        
                        title = f"{RefName}\n{intro_str}\nAggr mae = {round(mae,3)}\nIndi mae = {np.round(mae_model,3)}"    

                        
                        # plot the result
                        Plot_Result(data_df, title, Stock_Code, model = 'RNN' , name = f"Analysis/{RefName}" , IsSave = True, IsDisplay=False)

                       
                        # Save current result
                        Curr_Result = {
                            "id": id,
                            "Horizon": h,
                            "Data_Length":d,
                            "Layer": l,
                            "Train_Fraction": t,
                            "Seq_Len" : p,
                            "mae_aggr":mae,
                            "mae_indi":mae_model,
                            "RefName": RefName,    
                        }                        
    
                        Full_Result.append(Curr_Result)


                        
                        
                        # Save the Intermediate Result
                        if id % 10 == 0:                             
                            
                            Save_Prediction(Full_Result)
                            
                            Full_Result = []

                            print("Intermediate Result Saved")
                    

                        
                        # Clear Memory
                        del model
                        tf.keras.backend.clear_session()
                        gc.collect()


                        
                        # Next Model
                        id += 1                    
                    
                    else:

                        print(f"Skipped for {intro_str}")
                        
                        continue



Save_Prediction(Full_Result)

print("End Of Loop")

